In [1]:
%load_ext autoreload
%autoreload 2

In [22]:
import numpy as np
import morpher
import pandas as pd
import pandas_profiling
from collections import defaultdict
import math

In [3]:
df_mimic = pd.read_csv("~/cohorts/hs_mimic.csv",index_col=0)
df_sinai = pd.read_csv("~/cohorts/hs_sinai.csv",index_col=0)

/home/freith01/anaconda3/envs/fiber/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (11,78,79,81,82,84,85,178,184) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
xls = pd.ExcelFile('mapping.xls')
mapping = xls.parse(xls.sheet_names[1])
mapping.set_index('Field',inplace=True)

In [5]:
df_sinai_copy = df_sinai.copy()

In [6]:
df_sinai_copy.head()

,medical_record_number,age_in_days,date_of_birth,month_of_birth,gender,religion,race,patient_ethnic_group,deceased_indicator,mother_account_number,...,result_flag_pt_from_1_days_before_to_0_days_after,numeric_value_lactate_from_3_days_before_to_3_days_before,abnormal_flag_lactate_from_3_days_before_to_3_days_before,result_flag_lactate_from_3_days_before_to_3_days_before,numeric_value_lactate_from_2_days_before_to_2_days_before,abnormal_flag_lactate_from_2_days_before_to_2_days_before,result_flag_lactate_from_2_days_before_to_2_days_before,numeric_value_lactate_from_1_days_before_to_0_days_after,abnormal_flag_lactate_from_1_days_before_to_0_days_after,result_flag_lactate_from_1_days_before_to_0_days_after
0,160153144,21819,1952-01-01,October,Male,None,White,N,No,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.25,True,NaN
1,160153144,21826,1952-01-01,October,Male,None,White,N,No,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1772881649,345005,1066-01-01,August,Male,Catholic,Unknown,Non-Hispanic,No,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1772881649,345009,1066-01-01,August,Male,Catholic,Unknown,Non-Hispanic,No,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,331431889,28767,1930-01-01,September,Female,Unknown,Hispanic/Latino,NaN,No,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.18,True,NaN


In [7]:
df_sinai[['hypertension_complicated_precondition_from_inf_days_before_to_0_days_after', 'hypertension_uncomplicated_precondition_from_inf_days_before_to_0_days_after']]

,hypertension_complicated_precondition_from_inf_days_before_to_0_days_after,hypertension_uncomplicated_precondition_from_inf_days_before_to_0_days_after
0,False,True
1,False,True
2,False,True
3,False,True
4,False,True
5,False,True
6,False,True
7,True,True
8,False,True
9,False,True


In [8]:
merge_fields = defaultdict(lambda:[])
elixhauser = 0
series = defaultdict(lambda:[])

for _,row in df_sinai_copy.iterrows():
    for field in row.keys():        
        if mapping.loc[field,'Action'] == 'merge':
            merge_fields[mapping.loc[field,'Name']].append(row[field])
        if not pd.isna(mapping.loc[field,'Score']):
            elixhauser += int(row[field])*mapping.loc[field,'Score']            
    
    for mfield in merge_fields:
        series[mfield].append(any(merge_fields[mfield]))    
    series['ELIXHAUSER_SCORE'].append(elixhauser)
    
    merge_fields = defaultdict(lambda:[])
    elixhauser = 0    

for mfield in series:
    df_sinai_copy[mfield] = pd.Series(series[mfield])  
       

In [9]:
df_sinai_copy[['HYPERTENSION']]

,HYPERTENSION
0,True
1,True
2,True
3,True
4,True
5,True
6,True
7,True
8,True
9,True


In [10]:
for index, column in mapping.iterrows():
    if column['Action'] == 'remove' or column['Action'] == 'merge':
        df_sinai_copy.drop([index], axis=1, inplace=True)        
    if column['Action'] == 'rename':
        df_sinai_copy.rename(columns={ index : column['Name'] }, inplace=True)

In [11]:
df_sinai_copy.head(26)

,AGE_AT_ADMISSION,GENDER,CONGESTIVE_HEART_FAILURE,FLUID_ELECTROLYTE,LIVER_DISEASE,RHEUMATOID_ARTHRITIS,ALCOHOL_ABUSE,BLOOD_LOSS_ANEMIA,CARDIAC_ARRHYTHMIAS,CHRONIC_PULMONARY,...,PT_DTS_3.0,PT_DTS_2.0,PT_DTS_1.0,LACTATE_DTS_3.0,LACTATE_DTS_2.0,LACTATE_DTS_1.0,AKI,HYPERTENSION,HYPOTHYROIDISM,ELIXHAUSER_SCORE
0,21819,Male,False,False,False,False,False,False,False,False,...,NaN,NaN,15.950000,NaN,NaN,2.250000,False,True,False,-4.0
1,21826,Male,False,True,False,False,False,False,True,False,...,14.6,14.6,NaN,NaN,NaN,NaN,False,True,False,6.0
2,345005,Male,False,False,False,False,False,False,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,False,True,False,7.0
3,345009,Male,False,False,False,False,False,False,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,False,True,False,7.0
4,28767,Female,False,False,False,False,False,False,False,False,...,NaN,14.1,14.000000,NaN,NaN,1.180000,False,True,False,-1.0
5,28770,Female,False,False,False,False,False,False,False,False,...,14.1,NaN,NaN,1.180000,NaN,NaN,False,True,False,-1.0
6,20082,Female,False,False,False,False,False,False,False,False,...,NaN,NaN,15.500000,NaN,NaN,1.333333,False,True,False,-2.0
7,20090,Female,False,True,False,False,False,False,False,False,...,14.2,NaN,NaN,0.800000,NaN,NaN,False,True,False,6.0
8,26060,Male,False,False,False,False,False,False,False,False,...,NaN,NaN,13.600000,NaN,NaN,1.811111,False,True,False,0.0
9,26065,Male,False,False,False,False,False,False,False,False,...,NaN,NaN,NaN,0.633333,NaN,NaN,False,True,False,0.0


In [28]:
df_sinai_copy_copy = df_sinai_copy.copy()

In [29]:
df_sinai_copy_copy['AKI'].dtype
#df_sinai_copy_copy['HYPERTENSION'] = df_sinai_copy_copy['HYPERTENSION'].astype(bool)
#df_sinai_copy_copy['AKI'] = df_sinai_copy_copy['AKI'].astype(bool)

dtype('bool')

In [30]:
for column in df_sinai_copy_copy:
    print(f"{column} is type { df_sinai_copy_copy[column].dtype }")


AGE_AT_ADMISSION is type int64
GENDER is type object
CONGESTIVE_HEART_FAILURE is type bool
FLUID_ELECTROLYTE is type bool
LIVER_DISEASE is type bool
RHEUMATOID_ARTHRITIS is type bool
ALCOHOL_ABUSE is type bool
BLOOD_LOSS_ANEMIA is type bool
CARDIAC_ARRHYTHMIAS is type bool
CHRONIC_PULMONARY is type bool
COAGULOPATHY is type bool
DEFICIENCY_ANEMIAS is type bool
DEPRESSION is type bool
DIABETES_COMPLICATED is type bool
DIABETES_UNCOMPLICATED is type bool
DRUG_ABUSE is type bool
LYMPHOMA is type bool
METASTATIC_CANCER is type bool
OBESITY is type bool
OTHER_NEUROLOGICAL is type bool
PARALYSIS is type bool
PEPTIC_ULCER is type bool
PERIPHERAL_VASCULAR is type bool
PSYCHOSES is type bool
PULMONARY_CIRCULATION is type bool
RENAL_FAILURE is type bool
SOLID_TUMOR is type bool
VALVULAR_DISEASE is type bool
WEIGHT_LOSS is type bool
BUN_DTS_3.0 is type float64
BUN_DTS_2.0 is type float64
BUN_DTS_1.0 is type float64
BLD_CREA_DTS_3.0 is type float64
BLD_CREA_DTS_2.0 is type float64
BLD_CREA_DTS_1.0

In [31]:
df_sinai_copy_copy['AKI']

0        False
1        False
2        False
3        False
4        False
5        False
6        False
7        False
8        False
9        False
10       False
11       False
12       False
13       False
14       False
15       False
16       False
17       False
18       False
19       False
20       False
21       False
22       False
23       False
24        True
25        True
26        True
27       False
28       False
29       False
         ...  
25769    False
25770    False
25771    False
25772    False
25773    False
25774    False
25775    False
25776    False
25777    False
25778    False
25779    False
25780    False
25781    False
25782    False
25783    False
25784    False
25785    False
25786    False
25787    False
25788    False
25789    False
25790    False
25791    False
25792    False
25793    False
25794    False
25795    False
25796    False
25797    False
25798    False
Name: AKI, Length: 25799, dtype: bool

In [32]:
for column in df_sinai_copy_copy:
    if df_sinai_copy_copy[column].dtype == bool:
        df_sinai_copy_copy[column] = df_sinai_copy_copy[column].apply(lambda x: x if x == np.NaN else int(x))
    


In [33]:
df_sinai_copy_copy.head(26)

,AGE_AT_ADMISSION,GENDER,CONGESTIVE_HEART_FAILURE,FLUID_ELECTROLYTE,LIVER_DISEASE,RHEUMATOID_ARTHRITIS,ALCOHOL_ABUSE,BLOOD_LOSS_ANEMIA,CARDIAC_ARRHYTHMIAS,CHRONIC_PULMONARY,...,PT_DTS_3.0,PT_DTS_2.0,PT_DTS_1.0,LACTATE_DTS_3.0,LACTATE_DTS_2.0,LACTATE_DTS_1.0,AKI,HYPERTENSION,HYPOTHYROIDISM,ELIXHAUSER_SCORE
0,21819,Male,0,0,0,0,0,0,0,0,...,NaN,NaN,15.950000,NaN,NaN,2.250000,0,1,0,-4.0
1,21826,Male,0,1,0,0,0,0,1,0,...,14.6,14.6,NaN,NaN,NaN,NaN,0,1,0,6.0
2,345005,Male,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,0,1,0,7.0
3,345009,Male,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,0,1,0,7.0
4,28767,Female,0,0,0,0,0,0,0,0,...,NaN,14.1,14.000000,NaN,NaN,1.180000,0,1,0,-1.0
5,28770,Female,0,0,0,0,0,0,0,0,...,14.1,NaN,NaN,1.180000,NaN,NaN,0,1,0,-1.0
6,20082,Female,0,0,0,0,0,0,0,0,...,NaN,NaN,15.500000,NaN,NaN,1.333333,0,1,0,-2.0
7,20090,Female,0,1,0,0,0,0,0,0,...,14.2,NaN,NaN,0.800000,NaN,NaN,0,1,0,6.0
8,26060,Male,0,0,0,0,0,0,0,0,...,NaN,NaN,13.600000,NaN,NaN,1.811111,0,1,0,0.0
9,26065,Male,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,0.633333,NaN,NaN,0,1,0,0.0


In [34]:
df_sinai_copy_copy['AGE_AT_ADMISSION']

0         21819
1         21826
2        345005
3        345009
4         28767
5         28770
6         20082
7         20090
8         26060
9         26065
10       344958
11       344969
12        26276
13        26282
14        27286
15        27296
16        19381
17        19386
18        23385
19        23395
20        20547
21        20552
22        29918
23        29926
24        27883
25        27896
26        25185
27        25192
28       344870
29       344875
          ...  
25769     15661
25770     12138
25771     24452
25772    342670
25773     19798
25774    343003
25775     18922
25776     26124
25777     18592
25778     22112
25779     18119
25780    342830
25781    342772
25782     20949
25783     21832
25784    342666
25785     25482
25786    342679
25787     20623
25788     22643
25789     21026
25790     18597
25791     24662
25792     15631
25793     27360
25794     22253
25795     22352
25796    342416
25797     17981
25798     12321
Name: AGE_AT_ADMISSION, 

In [36]:
df_sinai_copy_copy['AGE_AT_ADMISSION'] = df_sinai_copy_copy['AGE_AT_ADMISSION'].apply(lambda x: math.ceil(x/365))

In [37]:
df_sinai_copy_copy['AGE_AT_ADMISSION'] 

0         60
1         60
2        946
3        946
4         79
5         79
6         56
7         56
8         72
9         72
10       946
11       946
12        72
13        73
14        75
15        75
16        54
17        54
18        65
19        65
20        57
21        57
22        82
23        82
24        77
25        77
26        69
27        70
28       945
29       945
        ... 
25769     43
25770     34
25771     67
25772    939
25773     55
25774    940
25775     52
25776     72
25777     51
25778     61
25779     50
25780    940
25781    940
25782     58
25783     60
25784    939
25785     70
25786    939
25787     57
25788     63
25789     58
25790     51
25791     68
25792     43
25793     75
25794     61
25795     62
25796    939
25797     50
25798     34
Name: AGE_AT_ADMISSION, Length: 25799, dtype: int64

In [39]:
df_sinai_copy_copy['AGE_AT_ADMISSION'] = df_sinai_copy_copy['AGE_AT_ADMISSION'].apply(lambda x: x if x <= 90 else 90)

In [42]:
df_sinai_copy_copy['AGE_AT_ADMISSION']

0        60
1        60
2        90
3        90
4        79
5        79
6        56
7        56
8        72
9        72
10       90
11       90
12       72
13       73
14       75
15       75
16       54
17       54
18       65
19       65
20       57
21       57
22       82
23       82
24       77
25       77
26       69
27       70
28       90
29       90
         ..
25769    43
25770    34
25771    67
25772    90
25773    55
25774    90
25775    52
25776    72
25777    51
25778    61
25779    50
25780    90
25781    90
25782    58
25783    60
25784    90
25785    70
25786    90
25787    57
25788    63
25789    58
25790    51
25791    68
25792    43
25793    75
25794    61
25795    62
25796    90
25797    50
25798    34
Name: AGE_AT_ADMISSION, Length: 25799, dtype: int64

In [44]:
df_sinai_copy_copy['GENDER']

0          Male
1          Male
2          Male
3          Male
4        Female
5        Female
6        Female
7        Female
8          Male
9          Male
10         Male
11         Male
12         Male
13         Male
14         Male
15         Male
16         Male
17         Male
18         Male
19         Male
20         Male
21         Male
22         Male
23         Male
24         Male
25         Male
26         Male
27         Male
28         Male
29         Male
          ...  
25769      Male
25770    Female
25771      Male
25772    Female
25773    Female
25774      Male
25775      Male
25776    Female
25777      Male
25778      Male
25779      Male
25780      Male
25781      Male
25782      Male
25783    Female
25784      Male
25785    Female
25786    Female
25787      Male
25788      Male
25789      Male
25790      Male
25791    Female
25792    Female
25793    Female
25794      Male
25795      Male
25796    Female
25797      Male
25798      Male
Name: GENDER, Length: 25

In [50]:
from sklearn.preprocessing import LabelBinarizer
from sklearn_pandas import DataFrameMapper

In [56]:
mapper = DataFrameMapper([('GENDER', LabelBinarizer())], df_out=True, default=None)

In [57]:
mapper.fit_transform(df_sinai_copy_copy.copy())

,GENDER,AGE_AT_ADMISSION,CONGESTIVE_HEART_FAILURE,FLUID_ELECTROLYTE,LIVER_DISEASE,RHEUMATOID_ARTHRITIS,ALCOHOL_ABUSE,BLOOD_LOSS_ANEMIA,CARDIAC_ARRHYTHMIAS,CHRONIC_PULMONARY,...,PT_DTS_3.0,PT_DTS_2.0,PT_DTS_1.0,LACTATE_DTS_3.0,LACTATE_DTS_2.0,LACTATE_DTS_1.0,AKI,HYPERTENSION,HYPOTHYROIDISM,ELIXHAUSER_SCORE
0,1,60.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,15.950000,NaN,NaN,2.250000,0.0,1.0,0.0,-4.0
1,1,60.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,14.6,14.6,NaN,NaN,NaN,NaN,0.0,1.0,0.0,6.0
2,1,90.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,0.0,7.0
3,1,90.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,0.0,7.0
4,0,79.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,14.1,14.000000,NaN,NaN,1.180000,0.0,1.0,0.0,-1.0
5,0,79.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,14.1,NaN,NaN,1.180000,NaN,NaN,0.0,1.0,0.0,-1.0
6,0,56.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,15.500000,NaN,NaN,1.333333,0.0,1.0,0.0,-2.0
7,0,56.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,14.2,NaN,NaN,0.800000,NaN,NaN,0.0,1.0,0.0,6.0
8,1,72.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,13.600000,NaN,NaN,1.811111,0.0,1.0,0.0,0.0
9,1,72.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,0.633333,NaN,NaN,0.0,1.0,0.0,0.0


In [ ]:
df_sinai_copy_copy.to_csv('~/cohorts/hs_sinai_preprocessed.csv',index=False)